In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import pandas as pd
from pyspark.sql.functions import translate
import pyspark.sql.functions as F
import re

In [2]:
spark=SparkSession.builder.appName("Market Analysis in Banking Domain").master("yarn").getOrCreate()
spark

### 1. Load data and create a Spark data frame

In [3]:
df = spark.read.format("csv").option("delimiter",';').option("inferSchema",True).option("header",True).load("Project 1_dataset_bank-full (2).csv")
df = df.withColumn('age', split(df['"age;""job""'], ';').getItem(0)).withColumn('job', split(df['"age;""job""'], ';').getItem(1))
df = df.select([F.col(col).alias(re.sub('""',"",col)) for col in df.columns])
df = df.withColumnRenamed('y"',"y")
df = df.drop('"age;job')
df = df.select(['age','job','marital','education','default','balance','housing','loan','contact','day','month','duration','campaign','pdays','previous','poutcome','y'])
df = df.withColumn("age",translate("age",'"','')).withColumn("job",translate("job",'"','')).withColumn("marital",translate("marital",'"','')).withColumn("education",translate("education",'"','')).withColumn("default",translate("default",'"','')).withColumn("housing",translate("housing",'"','')).withColumn("loan",translate("loan",'"','')).withColumn("contact",translate("contact",'"','')).withColumn("month",translate("month",'"','')).withColumn("poutcome",translate("poutcome",'"','')).withColumn('y',translate('y','"',''))
df = df.withColumn("age",col("age").cast("Integer"))
df.show(5)

+---+------------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
|age|         job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+------------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+---+
| 58|  management|married| tertiary|     no|   2143|    yes|  no|unknown|  5|  may|     261|       1|   -1|       0| unknown| no|
| 44|  technician| single|secondary|     no|     29|    yes|  no|unknown|  5|  may|     151|       1|   -1|       0| unknown| no|
| 33|entrepreneur|married|secondary|     no|      2|    yes| yes|unknown|  5|  may|      76|       1|   -1|       0| unknown| no|
| 47| blue-collar|married|  unknown|     no|   1506|    yes|  no|unknown|  5|  may|      92|       1|   -1|       0| unknown| no|
| 33|     unknown| single|  unknown|     no|      1|     no|  no|unknown|  5|  may|     19

## Creating a dataFrame into Sql dataset

In [4]:
df.createOrReplaceTempView("bank")
spark.conf.set("spark.sql.crossJoin.enabled", "true")

## 2. Give marketing success rate (No. of people subscribed / total no. of entries)

In [5]:
spark.sql("select (a.sub/b.tot)*100 as success from (select count(*) as sub from bank where  y='yes') a,(select count(*) as tot from bank) b").show()

+------------------+
|           success|
+------------------+
|11.698480458295547|
+------------------+



## 1. Give the maximum, mean, and minimum age of the average targeted customer

In [6]:
spark.sql('SELECT MAX(age) as Maximum_Targeted_Age, AVG(age) as Average_Targeted_Age,MIN(age) as Minimum_Targeted_Age FROM bank').show()

+--------------------+--------------------+--------------------+
|Maximum_Targeted_Age|Average_Targeted_Age|Minimum_Targeted_Age|
+--------------------+--------------------+--------------------+
|                  95|   40.93621021432837|                  18|
+--------------------+--------------------+--------------------+



## 2. Check the quality of customers by checking average balance, median balance of customers.

In [7]:
spark.sql('SELECT AVG(balance) as Average_Balance, percentile_approx(balance, 0.5) as Median_balance FROM bank').show()

+------------------+--------------+
|   Average_Balance|Median_balance|
+------------------+--------------+
|1362.2720576850766|           448|
+------------------+--------------+



## 3. Check if age matters in marketing subscription for deposit

In [8]:
spark.sql("select age, count(*) as number from bank where  y ='no' group by age order by number desc ").show()

+---+------+
|age|number|
+---+------+
| 32|  1864|
| 31|  1790|
| 33|  1762|
| 34|  1732|
| 35|  1685|
| 36|  1611|
| 30|  1540|
| 37|  1526|
| 39|  1344|
| 38|  1322|
| 40|  1239|
| 41|  1171|
| 42|  1131|
| 45|  1110|
| 43|  1058|
| 46|  1057|
| 44|  1043|
| 29|  1014|
| 47|   975|
| 48|   915|
+---+------+
only showing top 20 rows



## 4. Check if marital status mattered for a subscription to deposit

In [9]:
spark.sql("select marital, count(*) as number from bank where  y ='no' group by marital order by number desc ").show()

+--------+------+
| marital|number|
+--------+------+
| married| 24459|
|  single| 10878|
|divorced|  4585|
+--------+------+



## 5. Check if age and marital status together mattered for a subscription to deposit scheme.

In [10]:
spark.sql("select age, marital, count(*) as number from bank where y='no' group by age,marital order by number desc ").show()

+---+-------+------+
|age|marital|number|
+---+-------+------+
| 34|married|  1013|
| 33|married|   978|
| 35|married|   976|
| 36|married|   976|
| 37|married|   975|
| 32|married|   920|
| 31| single|   906|
| 39|married|   873|
| 30| single|   861|
| 40|married|   856|
| 45|married|   825|
| 38|married|   819|
| 32| single|   817|
| 41|married|   803|
| 31|married|   801|
| 46|married|   772|
| 42|married|   770|
| 43|married|   743|
| 47|married|   743|
| 44|married|   734|
+---+-------+------+
only showing top 20 rows



## 6. Do feature engineering for the bank and find the right age effect on the campaign.

In [11]:
df= df.withColumn("age_cat", when(df.age >65 ,"Old")
                                 .when(df.age <30,"Young")
                        .otherwise("Mid"))
df.groupBy("age_cat","y").count().show()

+-------+---+-----+
|age_cat|  y|count|
+-------+---+-----+
|    Mid| no|35146|
|  Young|yes|  928|
|  Young| no| 4345|
|    Mid|yes| 4041|
|    Old|yes|  320|
|    Old| no|  431|
+-------+---+-----+

